In [ ]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from substitutions import appos
from nltk.stem import PorterStemmer, WordNetLemmatizer
import copy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import numpy as np
import contractions
from nltk import RegexpTokenizer


In [ ]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir)
file_paths = [(data_dir + '/' + fname) for fname in file_names]
docID_to_doc = {}
for i in range(len(file_names)):
    docID_to_doc[i] = file_names[i]

In [ ]:
def remove_punct(tok):
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok
def remove_space(tok):
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):

    ftext = file_text.lower()
    # print(f"Original Text : \n{ftext}\n\n")
    # print("\n---------------------------------------\n")
    file_toks = word_tokenize(ftext)
    # print(f"After word_Tokenization : \n{file_toks}\n\n")
    # print("---------------------------------------")
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in file_toks if tok not in stop_words]
    # print(f"After removing stopwords : \n{file_toks}\n\n")
    # print("\n---------------------------------------\n")
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print("\n---------------------------------------\n")
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    return cleaned_toks

In [ ]:
def read_file(fpaths):
    
    file_tokens = []
    for fpath in fpaths:
        # print(fpath)
        f = open(fpath, 'r', encoding='utf-8', errors='ignore')
        ftxt_unprocessed = f.read()
        ftoks = preprocess_file(ftxt_unprocessed)
        file_tokens.append(ftoks)
    return file_tokens


In [ ]:
file_toks = read_file(file_paths)

In [ ]:
def create_positional_index(file_toks):
    pidx_dict = {}
    for i in range(len(file_toks)):
        for j in range(len(file_toks[i])):
            tok = file_toks[i][j]
            if(tok not in pidx_dict.keys()):
                pidx_dict[tok] = {}
                pidx_dict[tok][i] = [j]
            else:
                if(i in pidx_dict[tok].keys()):
                    pidx_dict[tok][i].append(j)
                else:
                    pidx_dict[tok][i] = [j]
    pidx_dict = dict(sorted(pidx_dict.items()))
    positional_index = {}
    terms = pidx_dict.keys()
    for t in terms:
        positional_index[t] = []
        for docID in pidx_dict[t].keys():
            term_doc_positions = copy.deepcopy(pidx_dict[t][docID])
            term_doc_positions.sort()
            positional_index[t].append([docID, term_doc_positions])        
        positional_index[t].sort(key=lambda x: x[0])
    return positional_index


In [38]:
positional_index = create_positional_index(file_toks)

In [ ]:
positional_index['changes']

In [60]:
def check_all_in_range(ptrs, ranges):
    for i in range(len(ptrs)):
        if(ptrs[i] >= ranges[i]):
            return False
    return True
# def process_phrase_query(positional_index, query_toks):

#     n_toks = len(query_toks)
#     outer_pointers = [0] * n_toks
#     outer_ranges = [len(positional_index[query_toks[i]]) for i in range(n_toks)]
#     while(check_all_in_range(outer_pointers, outer_ranges)):
#         all_equal = True
#         for i in range(n_toks - 1):
#             if(positional_index[query_toks[i]][outer_pointers[i]][0] != positional_index[query_toks[i + 1]][outer_pointers[i + 1]][0]):
#                 all_equal = False
#                 break
#         if(all_equal == True):
#             inner_pointers = [0] * n_toks
#             inner_ranges = [len(positional_index[query_toks[i]][outer_pointers[i]][1]) for i in range(n_toks)]
#             while(check_all_in_range(inner_pointers, inner_ranges)):
#                 for i in range(n_toks - 1):
                    
def process_phrase_query(positional_index, query_toks):

    n_toks = len(query_toks)
    print(query_toks)
    outer_pointers = [0] * n_toks
    # print(outer_pointers)
    outer_ranges = [len(positional_index[query_toks[i]]) for i in range(n_toks)]
    answer = []
    while(check_all_in_range(outer_pointers, outer_ranges)):
        all_equal = True
        for i in range(n_toks - 1):
            if(positional_index[query_toks[i]][outer_pointers[i]][0] != positional_index[query_toks[i + 1]][outer_pointers[i + 1]][0]):
                all_equal = False
                break
        if(all_equal == False):
            least_ptr = 0
            for i in range(1, n_toks):
                if(positional_index[query_toks[least_ptr]][outer_pointers[least_ptr]][0] > positional_index[query_toks[i]][outer_pointers[i]][0]):
                    least_ptr = i
            outer_pointers[least_ptr] += 1
        if(all_equal == True):

            
            # print(positional_index[query_toks[0]][outer_pointers[0]][0])
            # doc = positional_index[query_toks[0]][outer_pointers[0]][0]
            # j = 1
            # flag = True
            # inner_pointers = [0] * n_toks
            # inner_ranges = [len(positional_index[query_toks[i]][outer_pointers[i]][1]) for i in range(n_toks)]
            # print(inner_ranges)
            # # print(inner_ranges)
            # while(flag == True and check_all_in_range(inner_pointers, inner_ranges)):
            #     print(inner_pointers)
                
            #     cntr = 0
            #     for i in range(1, n_toks):
            #         # print(positional_index[query_toks[i]][outer_pointers[i]][1][inner_pointers[i]])
            #         if(positional_index[query_toks[i]][outer_pointers[i]][1][inner_pointers[i]] == positional_index[query_toks[i - 1]][outer_pointers[i - 1]][1][inner_pointers[i - 1]] + 1):
            #             cntr += 1
            #     if(cntr == n_toks - 1):
            #         flag = False
            #         answer.append(positional_index[query_toks[0]][outer_pointers[0]][0])
            #         break
     
            #     while((positional_index[query_toks[j]][outer_pointers[j]][1][inner_pointers[j]] <= positional_index[query_toks[j - 1]][outer_pointers[j - 1]][1][inner_pointers[j - 1]]) and inner_pointers[j] < inner_ranges[j]):
            #         inner_pointers[j] += 1
            #     if(inner_pointers[j] < inner_ranges[j]):
            #         if(positional_index[query_toks[j]][outer_pointers[j]][1][inner_pointers[j]] == positional_index[query_toks[j - 1]][outer_pointers[j - 1]][1][inner_pointers[j - 1]] + 1):
            #             j += 1
            #         else:
            #             j = 1
            #             inner_pointers[0] += 1
                
            # for i in range(n_toks):
            #     outer_pointers[i] += 1



In [36]:
# positional_index = {'water' : [[2, [2, 3, 4]], [10, [2, 3, 4, 5, 6]], [12, [5, 6, 7]], [13, [6, 8, 10]]], 'piano':[[2, []], [5, []]], 'happy': [[2, []]]}

In [61]:
x = process_phrase_query(positional_index, ["water", "piano", "happy"])


['water', 'piano', 'happy']
352
[1, 1, 4]
[0, 0, 0]
379
[1, 1, 1]
[0, 0, 0]


IndexError: list index out of range

In [47]:
print(positional_index['piano'])

[[21, [833]], [84, [1046]], [87, [1136]], [140, [328]], [153, [314]], [222, [855]], [241, [3370]], [252, [4341]], [273, [1449]], [295, [615]], [321, [181]], [338, [154]], [352, [2786]], [379, [2148]], [380, [115, 122]], [381, [93, 100]], [424, [323, 1880, 3553]], [426, [2524, 2534]], [482, [80]], [516, [142]], [519, [4446]], [520, [6694]], [573, [241]], [575, [333]], [590, [442]], [771, [2280]], [853, [1208]], [928, [502]], [941, [991, 1052]], [942, [111, 118]], [1003, [4]], [1004, [4]], [1078, [180]], [1104, [316, 322]]]
